In [1]:
using DataFrames, CSV
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots, GRUtils
using Distributions
using Gurobi, JuMP, Combinatorics

In [2]:
# Constants
N = 3 ; # number of passengers
K = 3 ;  # number of shuttles

In [3]:
# powerset(1:3, min=0, max=length(1:3))
a = 1:5
Combinatorics.powerset(a, 0, length(a)) 


Base.Iterators.Flatten{Vector{Base.Generator{Combinatorics.Combinations, Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}}}}(Base.Generator{Combinatorics.Combinations, Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}}[Base.Generator{Combinatorics.Combinations, Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}}(Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}(Combinatorics.var"#reorder#11"{UnitRange{Int64}}(1:5)), Combinatorics.Combinations(5, 0)), Base.Generator{Combinatorics.Combinations, Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}}(Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}(Combinatorics.var"#reorder#11"{UnitRange{Int64}}(1:5)), Combinatorics.Combinations(5, 1)), Base.Generator{Combinatorics.Combinations, Combinatorics.var"#10#13"{Combinatorics.var"#reorder#11"{UnitRange{Int64}}}}(Combinatorics.var"#10#13"{Combinat

In [4]:
# Use this cost later to load costs from BingsMap. For now, we will simply use the euclidian distance as shown below.
c = Matrix(DataFrame(CSV.File("../data/SimulatedCosts.csv")))
c = c[:, 2:size(c)[2]];

t = c  #let's consider in this initial model that the travel cost and the time and equivalent. 

8×8 Matrix{Float64}:
 1.2  4.0  1.2  4.5  1.2  0.9  1.2   0.9
 3.0  3.0  3.2  1.4  1.6  0.6  0.7   0.5
 1.4  5.0  4.6  1.9  3.8  3.0  0.4   0.2
 1.9  2.0  1.9  2.5  2.5  5.0  1.8   1.5
 2.5  1.0  2.4  1.2  6.2  2.0  1.3   1.7
 1.2  5.0  1.9  4.0  4.5  1.2  0.9   1.2
 4.0  2.0  2.5  2.0  2.1  2.4  6.5  18.3
 2.0  1.0  1.2  3.0  3.0  5.0  4.6   1.9

In [5]:
# Data
data = Matrix(DataFrame(CSV.File("../data/SimulatedDataTest.csv")));

q = data[1:2*N+2, 4];
d = data[1:2*N+2, 5] / 10;
e = data[1:2*N+2, 7];
l = data[1:2*N+2, 8];

#compute costs/time as euclidian distance.
c = zeros(2 * N + 2, 2 * N + 2)
for i = 1:2*N+2
    for j = 1:2*N+2
        c[i, j] = norm(data[i, 2:3] - data[j, 2:3]) / 10
    end
end

t = c  #let's consider in this initial model that the travel cost and the time and equivalent. 

8×8 Matrix{Float64}:
 0.0       0.291247  0.150659  0.743121  …  0.693661  1.31528   0.0
 0.291247  0.0       0.326118  0.620831     0.488037  1.03742   0.291247
 0.150659  0.326118  0.0       0.869122     0.789801  1.28146   0.150659
 0.743121  0.620831  0.869122  0.0          0.238253  1.26881   0.743121
 0.668374  0.379722  0.697821  0.585946     0.353577  0.733003  0.668374
 0.693661  0.488037  0.789801  0.238253  …  0.0       1.03335   0.693661
 1.31528   1.03742   1.28146   1.26881      1.03335   0.0       1.31528
 0.0       0.291247  0.150659  0.743121     0.693661  1.31528   0.0

# Two indices model

`The model is available here:` http://neumann.hec.ca/chairedistributique/common/pdptw.pdf

In [6]:
model = Model(Gurobi.Optimizer)

V =  1:2*N+2                     # all vertices
P = 2:N+1                        # pick up vertices
D = N+2:2*N+1                    # drop off vertices
PUD = 2:2*N+1                    # all pick up and drop off vertices together (all vertices except depot)

T = fill(1000, K)                # maximum duration of route k
L = 1000                         # maximum time of a drive
Q = 3                            # capacity of each car - IT HAS TO BE THE SAME FOR ALL SHUTTLES.

# Variables
@variable(model, u[1:2*N+2] >= 0)                      # u[i,k] time as which the vertex i is served by vehicle k (B in the paper)
@variable(model, x[1:2*N+2, 1:2*N+2] >= 0, Bin)        # x[i, j, k] = 1 if we go from node i to node j with vehicle k.  
@variable(model, r[1:N] >= 0)                          # r[i,k] travel time of passenger i (Note that passenger i is identified by) the vertex i+1
@variable(model, w[1:2*N+2] >= 0, Int)            # w[i,k] load of vehicle k when arriving upon leaving vertex i (Q in the paper)


# DEFINE SUBSET S WITH POWERSET
S = []


#############  Constraints  #############

@constraint(model, [j in PUD], sum(x[i, j] for i in V) == 1)
@constraint(model, [i in PUD], sum(x[i, j] for j in V) == 1)

# Define the precedence constraints
for sub in S
    @constraint(model, sum(sum(x[i, j] for i in sub) for j in sub) <= length(sub) - 2)
end

@constraint(model, [i in V, j in V, k = 1:K], u[j] >= (u[i] + d[i] + t[i, j]) * x[i, j])
@constraint(model, [i in V, j in V, k = 1:K], w[j] >= (w[i] + q[j]) * x[i, j])
@constraint(model, [i in V], e[i] <= u[i])
@constraint(model, [i in V], u[i] <= l[i])

@constraint(model, [i in V, k = 1:K], max(0, q[i]) <= w[i])
@constraint(model, [i in V, k = 1:K], w[i] <= min(Q, Q + q[i]))

# objective
@objective(model, Min, sum(sum(sum(c[i, j] * x[i, j]) for i in V) for j in V));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


In [7]:
function plot_route(df, x_val, w_val, plot_size)
    
    # df: dataframe containing all the input data:
    #     column 1: Passenger ID
    #     column 2: PickUp/DropOff Pos_x
    #     column 3: PickUp/DropOff Pos_y
    #     column 4: load (number of passengers to embark)
    #     column 5: service duration (embark time)
    #     column 6: preferred pick up time
    #     column 7: lower bound of pick up time
    #     column 8: upper bound of pick up time

    # car_nbr: the vehicle number to plot

    # x_val: the values of x
    
    #constants 
    N = round(Int,(size(df)[1]-2)/2)

    cars_used = findall([sum(x_val[:,:,i]) for i=1:size(x_val)[3]] .> 2)
    print(cars_used)
    
    println("Number of cars used: $(sum(cars_used))")
    println("The car(s) used are(is): ", cars_used)


    # plot the depot
    Plots.scatter([df[1, 2]], [df[2*N+2, 3]], 
        size = (plot_size[1], plot_size[2]),
        titlefont=font(20, "Computer Modern"), 
        color="black", 
        markersize=5, 
        labels="Depot", 
        title="Result for different cars",  
        legend = :outertopleft,
        format=:png)

    # plot all pick-up points
    for i=2:N+1
        pass_ID = round(Integer, df[i,1])

        Plots.scatter!([df[i, 2]], [df[i, 3]],
            color="lightgreen", 
            xlabel="x", ylabel="y",
            labels="PickUp ID: $pass_ID - $(data[i,4]) passenger(s)",  
            annotations = (df[i, 2], df[i, 3], Plots.text("  Pick:$pass_ID", :left, pointsize=10)),
            format=:png,
            markersize=5)

        Plots.scatter!([df[i+N, 2]], [df[i+N, 3]],
            color="red", 
            xlabel="x", ylabel="y",
            labels="DropOff ID: $pass_ID - $(data[i,4]) passenger(s)",  
            annotations = (df[i+N, 2], df[i+N, 3], Plots.text("  Drop:$pass_ID", :left, pointsize=10)),
            format=:png,
            markersize=5)
    end

    colors = ["black", "red", "green", "blue", "pink"]
    for car in cars_used
        vals = findall(x_val[:, :, car] .== 1)
        print(vals)
        for elem in vals
            i,j = elem[1], elem[2]
            print(i, " ", j, " --> ")
            Plots.plot!([df[i, 2], df[j, 2]], [df[i, 3], df[j, 3]], color=colors[car], labels="", textposition="bottom left", linewidth=1, arrow=true, arrowwidth=3, format=:png)
        end
        
        # for i in 1:2*N+2
        #     for j in 1:2*N+2
        #         if x_val[i, j, car] == 1
        #             print(i, " ", j, " --> ")
        #             Plots.plot!([df[i, 2], df[j, 2]], [df[i, 3], df[j, 3]], color=colors[car], labels="", textposition="bottom left", linewidth=1, arrow=true, arrowwidth=3, format=:png)
        #         end
        #     end
        # end
    end

    # plot the depot
    Plots.scatter!([df[1, 2]], [df[2*N+2, 3]], 
        titlefont=font(20, "Computer Modern"), 
        color="black", 
        markersize=5, 
        labels="Depot", 
        format=:png)

end


plot_route (generic function with 1 method)